# OCIMF Environmental Loading - Interactive Visualization Example

This notebook demonstrates the OCIMF Environmental Loading module with comprehensive charting capabilities.

## Features Demonstrated:
- Loading OCIMF coefficient database
- 2D interpolation of coefficients
- 3D surface plots
- Polar force diagrams
- Force vector visualizations
- Validation charts
- Interactive plotly charts

In [ ]:
# Import required libraries
import sys
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Add src to path if running from examples directory
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root / 'src'))

from marine_engineering.environmental_loading import (
    OCIMFDatabase,
    EnvironmentalForces,
    EnvironmentalConditions,
    VesselGeometry,
    create_sample_database,
)

# Set style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

print("Imports successful!")

## 1. Create Sample OCIMF Database

First, we'll create a sample database with multiple vessel types and loading conditions.

In [ ]:
# Create sample database
db_path = project_root / 'data' / 'ocimf' / 'ocimf_coefficients_sample.csv'
db_path.parent.mkdir(parents=True, exist_ok=True)

if not db_path.exists():
    create_sample_database(
        str(db_path),
        num_vessels=5,
        num_headings=13,
        num_displacements=3
    )

# Load database
db = OCIMFDatabase(str(db_path))

print(f"Database loaded: {len(db.data)} entries")
print(f"Vessel types: {db.data['vessel_type'].unique()}")
print(f"Heading range: {db.headings.min():.0f}° - {db.headings.max():.0f}°")
print(f"Displacement range: {db.displacements.min():,.0f} - {db.displacements.max():,.0f} tonnes")

# Preview data
db.data.head(10)

## 2. Coefficient Interpolation

Demonstrate 2D interpolation for arbitrary heading and displacement values.

In [ ]:
# Test interpolation at various points
test_cases = [
    (0, 250000, "Head-on, VLCC"),
    (45, 275000, "Quartering, VLCC (interpolated)"),
    (90, 300000, "Beam, VLCC"),
    (135, 250000, "Stern quarter, VLCC"),
    (45, 150000, "Quartering, Suezmax"),
]

results_df = []
for heading, displacement, description in test_cases:
    coeffs = db.get_coefficients(heading, displacement)
    results_df.append({
        'Description': description,
        'Heading (°)': heading,
        'Displacement (t)': f"{displacement:,}",
        'CXw': f"{coeffs.CXw:.3f}",
        'CYw': f"{coeffs.CYw:.3f}",
        'CMw': f"{coeffs.CMw:.3f}",
        'CXc': f"{coeffs.CXc:.3f}",
        'CYc': f"{coeffs.CYc:.3f}",
        'CMc': f"{coeffs.CMc:.3f}",
    })

pd.DataFrame(results_df)

## 3. 3D Surface Plots

Visualize how coefficients vary with heading and displacement.

In [ ]:
# Create output directory for charts
output_dir = project_root / 'examples' / 'outputs' / 'ocimf_charts'
output_dir.mkdir(parents=True, exist_ok=True)

# Generate 3D surface plots for key coefficients
coefficients_to_plot = ['CXw', 'CYw', 'CMw', 'CXc', 'CYc', 'CMc']

for coef in coefficients_to_plot[:3]:  # Wind coefficients first
    print(f"Generating 3D surface plot for {coef}...")
    db.plot_coefficient_surface(
        coef,
        save_path=str(output_dir / f"surface_{coef}.png"),
        interactive=False
    )
    
print("\n3D surface plots saved to:", output_dir)

In [ ]:
# Display one of the generated plots
from IPython.display import Image, display

display(Image(filename=str(output_dir / 'surface_CYw.png')))

## 4. Polar Diagrams

Show wind force coefficients as polar plots for different vessel displacements.

In [ ]:
# Generate polar diagrams for different displacements
displacements_to_plot = [250000, 300000, 150000]

for disp in displacements_to_plot:
    print(f"Generating polar diagram for {disp:,} tonnes...")
    db.plot_polar_diagram(
        displacement=disp,
        save_path=str(output_dir / f"polar_{disp}.png")
    )

print("\nPolar diagrams saved to:", output_dir)

In [ ]:
# Display polar diagram
display(Image(filename=str(output_dir / 'polar_250000.png')))

## 5. Interpolation Quality Analysis

Evaluate the quality of interpolation by comparing database points with interpolated surface.

In [ ]:
# Generate interpolation quality heatmaps
for coef in ['CYw', 'CXw']:
    print(f"Generating interpolation quality heatmap for {coef}...")
    db.plot_interpolation_quality(
        coefficient_name=coef,
        save_path=str(output_dir / f"interp_quality_{coef}.png")
    )

display(Image(filename=str(output_dir / 'interp_quality_CYw.png')))

## 6. Environmental Force Calculations

Calculate actual forces on a vessel under specific environmental conditions.

In [ ]:
# Create force calculator
calc = EnvironmentalForces(db)

# Define environmental conditions
conditions = EnvironmentalConditions(
    wind_speed=25.0,  # m/s (about 49 knots - severe storm)
    wind_direction=60.0,  # degrees (quartering wind)
    current_speed=2.0,  # m/s (about 4 knots - strong current)
    current_direction=45.0,  # degrees
    air_density=1.225,  # kg/m³
    water_density=1025.0  # kg/m³
)

# Define vessel geometry (VLCC)
geometry = VesselGeometry(
    loa=330.0,  # meters
    beam=60.0,  # meters
    draft=22.0  # meters
)

displacement = 300000  # tonnes

# Calculate forces
results = calc.calculate_total_forces(conditions, geometry, displacement)

# Display results
print("="*60)
print("ENVIRONMENTAL FORCE CALCULATION RESULTS")
print("="*60)
print(f"\nVessel: VLCC, {displacement:,} tonnes")
print(f"LOA: {geometry.loa}m, Beam: {geometry.beam}m, Draft: {geometry.draft}m")
print(f"\nConditions:")
print(f"  Wind: {conditions.wind_speed} m/s @ {conditions.wind_direction}°")
print(f"  Current: {conditions.current_speed} m/s @ {conditions.current_direction}°")
print(f"\nWind Forces:")
print(f"  Fx (Longitudinal): {results.wind_fx/1e3:>10.1f} kN")
print(f"  Fy (Lateral):      {results.wind_fy/1e3:>10.1f} kN")
print(f"  Mz (Yaw):          {results.wind_mz/1e6:>10.1f} MN·m")
print(f"\nCurrent Forces:")
print(f"  Fx (Longitudinal): {results.current_fx/1e3:>10.1f} kN")
print(f"  Fy (Lateral):      {results.current_fy/1e3:>10.1f} kN")
print(f"  Mz (Yaw):          {results.current_mz/1e6:>10.1f} MN·m")
print(f"\nTotal Forces:")
print(f"  Fx (Longitudinal): {results.total_fx/1e3:>10.1f} kN")
print(f"  Fy (Lateral):      {results.total_fy/1e3:>10.1f} kN")
print(f"  Mz (Yaw):          {results.total_mz/1e6:>10.1f} MN·m")
print(f"\nResultant Force:   {np.sqrt(results.total_fx**2 + results.total_fy**2)/1e3:>10.1f} kN")
print(f"Force Angle:       {np.degrees(np.arctan2(results.total_fy, results.total_fx)):>10.1f}°")
print("="*60)

## 7. Force Vector Diagram

Visualize the force components as vectors.

In [ ]:
# Generate force diagram
calc.plot_force_diagram(
    results,
    save_path=str(output_dir / 'force_diagram.png')
)

display(Image(filename=str(output_dir / 'force_diagram.png')))

## 8. Parametric Study: Wind Speed Variation

Analyze how forces change with varying wind speeds.

In [ ]:
# Parametric study: wind speed variation
wind_speeds = np.linspace(5, 35, 15)  # 5 to 35 m/s
fx_values = []
fy_values = []
mz_values = []

for ws in wind_speeds:
    cond = EnvironmentalConditions(
        wind_speed=ws,
        wind_direction=60.0,
        current_speed=2.0,
        current_direction=45.0
    )
    res = calc.calculate_total_forces(cond, geometry, displacement)
    fx_values.append(res.total_fx / 1e3)
    fy_values.append(res.total_fy / 1e3)
    mz_values.append(res.total_mz / 1e6)

# Plot results
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

ax1.plot(wind_speeds, fx_values, 'b-o', linewidth=2, markersize=6, label='Fx (Longitudinal)')
ax1.plot(wind_speeds, fy_values, 'r-s', linewidth=2, markersize=6, label='Fy (Lateral)')
ax1.set_xlabel('Wind Speed (m/s)', fontsize=11)
ax1.set_ylabel('Force (kN)', fontsize=11)
ax1.set_title('Total Force vs Wind Speed', fontsize=12)
ax1.legend(fontsize=9)
ax1.grid(True, alpha=0.3)

ax2.plot(wind_speeds, mz_values, 'g-^', linewidth=2, markersize=6)
ax2.set_xlabel('Wind Speed (m/s)', fontsize=11)
ax2.set_ylabel('Moment (MN·m)', fontsize=11)
ax2.set_title('Yaw Moment vs Wind Speed', fontsize=12)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(output_dir / 'parametric_wind_speed.png', dpi=300, bbox_inches='tight')
plt.show()

## 9. Validation Against Excel

Compare Python calculations with Excel reference values.

In [ ]:
# Simulate Excel results (would normally come from Excel file)
# Adding small differences for demonstration
excel_results = {
    'wind_fx': results.wind_fx * 1.02,      # 2% difference
    'wind_fy': results.wind_fy * 0.98,      # -2% difference
    'wind_mz': results.wind_mz * 1.01,      # 1% difference
    'current_fx': results.current_fx * 1.03,  # 3% difference
    'current_fy': results.current_fy * 0.97,  # -3% difference
    'current_mz': results.current_mz * 1.02   # 2% difference
}

# Generate validation chart
calc.plot_validation_chart(
    excel_results,
    results,
    save_path=str(output_dir / 'validation_chart.png')
)

display(Image(filename=str(output_dir / 'validation_chart.png')))

## 10. Multi-Vessel Comparison

Compare forces on different vessel types under the same conditions.

In [ ]:
# Define different vessel types
vessels = [
    {'name': 'VLCC', 'loa': 330, 'beam': 60, 'draft': 22, 'disp': 300000},
    {'name': 'Suezmax', 'loa': 275, 'beam': 48, 'draft': 16, 'disp': 180000},
    {'name': 'Aframax', 'loa': 245, 'beam': 42, 'draft': 14, 'disp': 120000},
]

comparison_results = []

for vessel in vessels:
    geom = VesselGeometry(loa=vessel['loa'], beam=vessel['beam'], draft=vessel['draft'])
    res = calc.calculate_total_forces(conditions, geom, vessel['disp'])
    
    comparison_results.append({
        'Vessel': vessel['name'],
        'Displacement (t)': f"{vessel['disp']:,}",
        'LOA (m)': vessel['loa'],
        'Total Fx (kN)': f"{res.total_fx/1e3:.1f}",
        'Total Fy (kN)': f"{res.total_fy/1e3:.1f}",
        'Total Mz (MN·m)': f"{res.total_mz/1e6:.1f}",
        'Resultant (kN)': f"{np.sqrt(res.total_fx**2 + res.total_fy**2)/1e3:.1f}"
    })

comparison_df = pd.DataFrame(comparison_results)
print("\nMulti-Vessel Force Comparison")
print("="*80)
comparison_df

In [ ]:
# Visualize comparison
fig, ax = plt.subplots(figsize=(10, 6))

vessel_names = [v['name'] for v in vessels]
fx_vals = [float(r['Total Fx (kN)']) for r in comparison_results]
fy_vals = [float(r['Total Fy (kN)']) for r in comparison_results]
mz_vals = [float(r['Total Mz (MN·m)']) for r in comparison_results]

x = np.arange(len(vessel_names))
width = 0.25

ax.bar(x - width, fx_vals, width, label='Fx (Long.)', alpha=0.8)
ax.bar(x, fy_vals, width, label='Fy (Lat.)', alpha=0.8)
ax.bar(x + width, [m*1000 for m in mz_vals], width, label='Mz (kN·m)', alpha=0.8)

ax.set_ylabel('Force (kN)', fontsize=11)
ax.set_title('Environmental Forces Comparison by Vessel Type', fontsize=12)
ax.set_xticks(x)
ax.set_xticklabels(vessel_names)
ax.legend(fontsize=9)
ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig(output_dir / 'vessel_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

## Summary

This notebook has demonstrated:

1. ✅ Loading and managing OCIMF coefficient databases
2. ✅ 2D interpolation for arbitrary heading/displacement values
3. ✅ 3D surface visualization of coefficient distributions
4. ✅ Polar diagrams for wind force coefficients
5. ✅ Interpolation quality assessment
6. ✅ Environmental force calculations (wind + current)
7. ✅ Force vector diagrams
8. ✅ Parametric studies (wind speed variation)
9. ✅ Validation charts (Python vs Excel)
10. ✅ Multi-vessel comparisons

All generated charts are saved to: `examples/outputs/ocimf_charts/`

In [ ]:
# List all generated charts
import os

print("\nGenerated Charts:")
print("="*60)
for file in sorted(output_dir.glob('*.png')):
    size_kb = file.stat().st_size / 1024
    print(f"  {file.name:<40} ({size_kb:>6.1f} KB)")
print("="*60)